In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import shapes
import os
import requests
import zipfile

In [2]:
file_path = './heat_risk_analysis_Day+1_20240723.geoparquet'
gdf = gpd.read_parquet(file_path)

In [3]:
gdf

geometry  raster_value  \
0      POLYGON ((-106.44867 49.0167, -106.44867 48.98...           2.0   
1      POLYGON ((-123.08047 49.00174, -123.08047 48.9...           1.0   
2      POLYGON ((-111.44505 49.00174, -111.44505 48.9...           2.0   
3      POLYGON ((-111.14846 49.00174, -111.14846 48.9...           2.0   
4      POLYGON ((-106.19771 49.00174, -106.19771 48.9...           3.0   
...                                                  ...           ...   
13657  POLYGON ((-81.60363 24.61704, -81.60363 24.596...           2.0   
13658  POLYGON ((-81.76333 24.61704, -81.76333 24.596...           3.0   
13659  POLYGON ((-81.78615 24.59629, -81.78615 24.534...           2.0   
13660  POLYGON ((-81.80896 24.61704, -81.80896 24.575...           3.0   
13661  POLYGON ((-81.76333 24.5548, -81.76333 24.5340...           3.0   

       weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  \
0          1.209867e+09          1112119.0                30.0   
1          1.265276e+07          1100110.0                53.0   
2          4.662729e+08         10084945.0                30.0   
3          1.026204e+09           164299.0                30.0   
4          2.025355e+08           523544.0                30.0   
...                 ...                ...                 ...   
13657      2.006446e+08           352196.0                51.0   
13658      2.979025e+07            98644.0                51.0   
13659      1.930206e+08          1044932.0                51.0   
13660      2.791815e+08         27144432.0                51.0   
13661      2.791815e+08         27144432.0                51.0   

       weighted_GEOID10  weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  \
0             3059250.0                   0.0         198.0             20.0   
1             5398281.0                   0.0        1314.0             91.0   
2             3059484.0                   0.0         111.0             20.0   
3             3059260.0                   0.0         110.0             20.0   
4             3059256.0                   0.0          59.0           -999.0   
...                 ...                   ...           ...              ...   
13657         5122920.0                   0.0        4186.0             93.0   
13658         5122943.0                   0.0         444.0             91.0   
13659         5122980.0                   0.0       31451.0             83.0   
13660         5123024.0                   0.0        8120.0             87.0   
13661         5123024.0                   0.0        8120.0             87.0   

       weighted_F_HRI  ...  weighted_OVERALL_RANK  mode_ZCTA5CE20  \
0                 0.5  ...                 0.5660           59250   
1                 1.0  ...                 0.6419           98281   
2                 0.5  ...                 0.6977           59484   
3                 0.5  ...                 0.3299           59260   
4              -999.0  ...                 0.5314           59256   
...               ...  ...                    ...             ...   
13657             1.0  ...                 0.1901           22920   
13658             1.0  ...                 0.3487           22943   
13659             1.0  ...                 0.8069           22980   
13660             1.0  ...                 0.1912           23024   
13661             1.0  ...                 0.1912           23024   

       mode_AFFGEOID20  mode_GEOID20  mode_NAME20  mode_LSAD20  mode_STATE  \
0       860Z200US59250         59250        59250           Z5          MT   
1       860Z200US98281         98281        98281           Z5          WA   
2       860Z200US59484         59484        59484           Z5          MT   
3       860Z200US59260         59260        59260           Z5          MT   
4       860Z200US59256         59256        59256           Z5          MT   
...                ...           ...          ...          ...         ...   
13657   860Z200US22920        

In [4]:
gdf.columns

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'

In [20]:
file_1 = './data/heat_risk_analysis_Day 1_20240731_with_county_state.parquet'
gdf_1 = gpd.read_parquet(file_1)

In [22]:
gdf_1.head()

geometry  raster_value  \
0  POLYGON ((-95.10984 49.35965, -95.10984 49.344...           4.0   
1  POLYGON ((-95.01858 49.35965, -95.01858 49.344...           4.0   
2  POLYGON ((-94.92732 49.34479, -94.92732 49.329...           2.0   
3  POLYGON ((-94.92732 49.32993, -94.92732 49.300...           3.0   
4  POLYGON ((-95.13265 49.35965, -95.13265 49.240...           3.0   

   weighted_ALAND20  weighted_AWATER20  weighted_STATEFP10  weighted_GEOID10  \
0       323659886.0       1.219293e+09                27.0         2756711.0   
1       323659886.0       1.219293e+09                27.0         2756711.0   
2       323659886.0       1.219293e+09                27.0         2756711.0   
3       323659886.0       1.219293e+09                27.0         2756711.0   
4       323659886.0       1.219293e+09                27.0         2756711.0   

   weighted_MULTI_STATE  weighted_POP  weighted_PR_HRI  weighted_F_HRI  ...  \
0                   0.0          96.0           -999.0          -999.0  ...   
1                   0.0          96.0           -999.0          -999.0  ...   
2                   0.0          96.0           -999.0          -999.0  ...   
3                   0.0          96.0           -999.0          -999.0  ...   
4                   0.0          96.0           -999.0          -999.0  ...   

   mode_STATE_NAME  mode_LSAD_1  mode_STATEFP_2  mode_STATENS  \
0        Minnesota           06              27      00662849   
1        Minnesota           06              27      00662849   
2        Minnesota           06              27      00662849   
3        Minnesota           06              27      00662849   
4        Minnesota           06              27      00662849   

   mode_AFFGEOID_2  mode_GEOID_2  mode_STUSPS_2  mode_NAME_2  mode_LSAD_2  \
0      0400000US27            27             MN    Minnesota           00   
1      0400000US27            27             MN    Minnesota           00   
2      0400000US27            27             MN    Minnesota           00   
3      0400000US27            27             MN    Minnesota           00   
4      0400000US27            27             MN    Minnesota           00   

   highlight  
0       True  
1       True  
2       True  
3       True  
4       True  

[5 rows x 106 columns]

In [25]:
gdf_1.columns[:50]

Index(['geometry', 'raster_value', 'weighted_ALAND20', 'weighted_AWATER20',
       'weighted_STATEFP10', 'weighted_GEOID10', 'weighted_MULTI_STATE',
       'weighted_POP', 'weighted_PR_HRI', 'weighted_F_HRI', 'weighted_LOW_EMS',
       'weighted_P_NEHD', 'weighted_PR_NEHD', 'weighted_HHB_SCORE',
       'weighted_HHB_RANK', 'weighted_P_CHD', 'weighted_PR_CHD',
       'weighted_F_CHD', 'weighted_P_OBS', 'weighted_PR_OBS', 'weighted_F_OBS',
       'weighted_P_DIABETES', 'weighted_PR_DIABETES', 'weighted_F_DIABETES',
       'weighted_P_COPD', 'weighted_PR_COPD', 'weighted_F_COPD',
       'weighted_P_ASTHMA', 'weighted_PR_ASTHMA', 'weighted_F_ASTHMA',
       'weighted_P_MNTHL', 'weighted_PR_MNTHL', 'weighted_F_MNTHL',
       'weighted_F_SEN_COUNT', 'weighted_SEN_SCORE', 'weighted_SEN_RANK',
       'weighted_P_UNINSUR', 'weighted_PR_UNINSUR', 'weighted_P_POV',
       'weighted_PR_POV', 'weighted_P_UNEMP', 'weighted_PR_UNEMP',
       'weighted_P_NOHSDP', 'weighted_PR_NOHSDP', 'weighted_P_ISO'